In [1]:
import warnings
warnings.filterwarnings('ignore')


import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.applications import resnet50
from keras.models import Model, load_model
from keras.layers import Input
from tensorflow.keras.layers import BatchNormalization
import matplotlib.pyplot as plt
import cv2



2024-03-17 00:30:20.586120: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-17 00:30:20.673731: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-17 00:30:20.673827: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-17 00:30:20.675695: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-17 00:30:20.687591: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-17 00:30:20.689635: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
for dirname, _, filenames in os.walk('./data'):
    print(dirname)

./data
./data/Uninfected
./data/Parasitized
./data/cell_images
./data/cell_images/Uninfected
./data/cell_images/Parasitized


In [3]:
def data_prep(parasitized, uninfected):
    
    parasitized = ["./data/Parasitized" + '/' +  parasitize for parasitize in parasitized]
    uninfected = ["./data/Uninfected" + '/' +  uninfect for uninfect in uninfected]

    labels = len(parasitized) * ['parasitized'] + len(uninfected) * ['uninfected']
    data = parasitized + uninfected

    return pd.DataFrame({'Image_Path': data , 'Labels': labels})

In [4]:
df = data_prep(os.listdir("./data/Parasitized/"), os.listdir("./data/Uninfected/"))

In [5]:


print("Shape of the DataFrame:", df.shape)
print("\nHead of the DataFrame:")
print(df.head())
print("\nTail of the DataFrame:")
print(df.tail())



Shape of the DataFrame: (27558, 2)

Head of the DataFrame:
                                          Image_Path       Labels
0  ./data/Parasitized/C137P98ThinF_IMG_20151005_1...  parasitized
1  ./data/Parasitized/C133P94ThinF_IMG_20151004_1...  parasitized
2  ./data/Parasitized/C39P4thinF_original_IMG_201...  parasitized
3  ./data/Parasitized/C70P31_ThinF_IMG_20150819_1...  parasitized
4  ./data/Parasitized/C132P93ThinF_IMG_20151004_1...  parasitized

Tail of the DataFrame:
                                              Image_Path      Labels
27553  ./data/Uninfected/C145P106ThinF_IMG_20151016_1...  uninfected
27554  ./data/Uninfected/C72P33_ThinF_IMG_20150815_10...  uninfected
27555  ./data/Uninfected/C101P62ThinF_IMG_20150918_15...  uninfected
27556  ./data/Uninfected/C7NthinF_IMG_20150611_105444...  uninfected
27557  ./data/Uninfected/C183P144NThinF_IMG_20151201_...  uninfected


In [6]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Function to load and preprocess images
def load_images(file_paths, target_size=(64, 64)):
    images = []
    for path in file_paths:
        img = cv2.imread(path)
        img = cv2.resize(img, target_size)
        images.append(img)
    return np.array(images)

# Load and preprocess images
X = load_images(df['Image_Path'])
y = df['Labels']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build an improved CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())  
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(256, activation='relu'))  
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))  
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))  
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the CNN model
model.fit(X_train, (y_train == 'parasitized').astype(int), epochs=30, batch_size=32, validation_split=0.2)

# Extract features from the CNN model
cnn_features_train = model.predict(X_train)
cnn_features_test = model.predict(X_test)

# Reshape features for SVM
cnn_features_train_flat = cnn_features_train.reshape(cnn_features_train.shape[0], -1)
cnn_features_test_flat = cnn_features_test.reshape(cnn_features_test.shape[0], -1)

# List of kernel types
kernels = ['linear', 'poly', 'rbf']

# Train and evaluate SVM models for each kernel
for kernel_type in kernels:
    # Build an SVM model
    svm_model = SVC(kernel=kernel_type)
    svm_model.fit(cnn_features_train_flat, (y_train == 'parasitized'))

    # Predict using the SVM model
    svm_predictions = svm_model.predict(cnn_features_test_flat)

    # Evaluate the SVM model for each kernel
    accuracy = accuracy_score((y_test == 'parasitized'), svm_predictions)
    print(f"Accuracy of the SVM model (kernel: {kernel_type}): {accuracy*100}")


2024-03-17 00:30:54.128238: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Epoch 1/30
552/552 [==============================] - 115s 200ms/step - loss: 0.4710 - accuracy: 0.7828 - val_loss: 0.1702 - val_accuracy: 0.9392
Epoch 2/30
552/552 [==============================] - 130s 235ms/step - loss: 0.2040 - accuracy: 0.9335 - val_loss: 0.1760 - val_accuracy: 0.9535
Epoch 3/30
552/552 [==============================] - 127s 231ms/step - loss: 0.1712 - accuracy: 0.9451 - val_loss: 0.1775 - val_accuracy: 0.9433
Epoch 4/30
552/552 [==============================] - 127s 230ms/step - loss: 0.1538 - accuracy: 0.9508 - val_loss: 0.1571 - val_accuracy: 0.9540
Epoch 5/30
552/552 [==============================] - 106s 193ms/step - loss: 0.1402 - accuracy: 0.9541 - val_loss: 0.1449 - val_accuracy: 0.9544
Epoch 6/30
552/552 [==============================] - 103s 187ms/step - loss: 0.1338 - accuracy: 0.9563 - val_loss: 0.1510 - val_accuracy: 0.9578
Epoch 7/30
552/552 [==============================] - 102s 184ms/step - loss: 0.1231 - accuracy: 0.9595 - val_loss: 0.1572 -